In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

###Additional imports
import pickle
from sklearn.metrics.pairwise import pairwise_distances
from future.utils import iteritems
import gc
import time 

/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl


In [2]:
## Read in the pickled file
#filename='../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
#infile = open(filename,'rb')
#glove_vector = pickle.load(infile)
#infile.close()

In [3]:
def find_analogies(w1,w2,w3,wnn,dist_metric):
    
    filename='../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
    infile = open(filename,'rb')
    word_embedding = pickle.load(infile)
    infile.close()
    
  
    for w in (w1, w2, w3 , wnn):
        ## If any words are not there exit
        if w not in word_embedding:
            print("%s not in dictionary" % w)
            return
        ## Store the vectors 
    v1 = word_embedding[w1]
    v2 = word_embedding[w2]
    v3 = word_embedding[w3]
    vnn = word_embedding[wnn]
   
    min_dist = float('inf')
    best_word = ''
    
    ## appending till 300 as certain vectors in the word emdeddings have less than 300 featues
   
    v1 = np.expand_dims(v1,axis=-1)
    v2 = np.expand_dims(v2,axis=-1)
    v3 = np.expand_dims(v3,axis=-1)
    vnn = np.expand_dims(vnn,axis=-1)
    #print(np.shape(v1))
    v1 = np.append(v1,np.zeros([300-np.shape(v1)[0],1]),axis=0)
    v2 = np.append(v2,np.zeros([300-np.shape(v2)[0],1]),axis=0)
    v3 = np.append(v3,np.zeros([300-np.shape(v3)[0],1]),axis=0)
    vnn = np.append(vnn,np.zeros([300-np.shape(vnn)[0],1]),axis=0)
    v_calc = v1-v2+v3
    v_calc_trans = np.transpose(v_calc)
    #v_calc = np.expand_dims(v_calc,axis=-1)
    vnn_trans = np.transpose(vnn)
    
    nn_dict={}
    
    for word,v4 in iteritems(word_embedding):
        v4 = np.expand_dims(v4,axis=-1)
        v4 = np.append(v4,np.zeros([300-np.shape(v4)[0],1]),axis=0)
        
        if dist_metric == 'cosine':
            v4=np.transpose(v4)
            d=1 - v4.dot(v_calc) / (np.linalg.norm(v4) * np.linalg.norm(v_calc_trans))
            dnn=1 - v4.dot(vnn) / (np.linalg.norm(v4) * np.linalg.norm(vnn_trans))
            nn_dict[word]=dnn
        else: 
            d=np.linalg.norm(v_calc - v4)
            dnn=np.linalg.norm(vnn - v4)
            nn_dict[word]=dnn
            
        
        if d < min_dist:
            min_dist = d
            best_word=word
        
    
    nn_sorted_list = sorted(nn_dict.items(), key=lambda x: x[1])
    nn_words = nn_sorted_list[0:11]
    
    del word_embedding
    
    return best_word,nn_words
    ###########PART COMMENTED DUE TO LACK OF RAM ####################
    
    #for x in word_embedding.values():
     #   a = np.expand_dims(x,axis=0)
      #  b = np.append(a,np.zeros([1,300-np.shape(a)[1]]),axis=1)     
      #  vec_list.append(b)
        
    ##del word_embedding
    ##gc.collect()
    #vec_numpy = np.asarray(vec_list) 
    #del vec_list
    #gc.collect()
    #vec_numpy=np.squeeze(vec_numpy)
    #print(np.shape(vec_numpy))        
    
    #distances = pairwise_distances(v_calc,vec_numpy,metric=dist_metric)
    #print(np.shape(distances))
    
    ###########PART COMMENTED DUE TO LACK OF RAM ####################
    

In [4]:
best_word,nn_words=find_analogies(w1='king',w2='man',w3='queen',wnn='India',dist_metric='euclidean')

In [5]:
best_word

'queen'

In [6]:
nn_words

[('India', 0.0),
 ('Delhi', 5.088391630961377),
 ('Mumbai', 5.517324970021202),
 ('Indian', 5.859656560200182),
 ('Bangalore', 5.879491541417833),
 ('Pakistan', 5.913844647708191),
 ('Chennai', 5.942228246023854),
 ('Lanka', 5.9832402745300435),
 ('Kolkata', 6.062841568821642),
 ('Hyderabad', 6.089306673392133),
 ('Bangladesh', 6.126212599676059)]

In [7]:
best_word,nn_words=find_analogies(w1='happy',w2='hapiness',w3='sad',wnn='Japan',dist_metric='cosine')

In [8]:
best_word

'sad'

In [9]:
nn_words

[('Japan', array([[-2.22044605e-16]])),
 ('Tokyo', array([[0.21443195]])),
 ('Japanese', array([[0.23275325]])),
 ('Korea', array([[0.297462]])),
 ('Taiwan', array([[0.3441073]])),
 ('Osaka', array([[0.35745652]])),
 ('China', array([[0.37097326]])),
 ('Asia', array([[0.40073338]])),
 ('JAPAN', array([[0.40842062]])),
 ('japan', array([[0.40863329]])),
 ('Kyoto', array([[0.4318104]]))]